In [357]:
import numpy as np
import pandas as pd
import itertools
from numba import jit, prange
import json

try:
    df # does a exist in the current namespace
except NameError:
#     dfa = pd.read_csv(r'smoking_driking_dataset_Ver01.csv')
#     dfa["sex2"] = [0 if dfa["sex"][i] == "Male" else 1 for i in range(dfa.shape[0])]
#     dfa["drink2"] = [0 if dfa["DRK_YN"][i] == "N" else 1 for i in range(dfa.shape[0])]
#     df = dfa[['sex2', 'age', 'tot_chole', 'drink2']][0:5000].copy()
    
    dfa = pd.DataFrame(np.random.random((5000,4)), columns=['Column 1', 'Column 2', 'Column 3', 'Column 4'])
    df = dfa.copy()
    
    
    catvalues = [2, 3, 3, 2]
    
try:
    G # does a exist in the current namespace
except NameError:
    G = [[i+1 for i in range(df.shape[1])]]
    
try:
    E # does a exist in the current namespace
except NameError:
    E = [[i,j] for i in range(1, df.shape[1]) for j in range(i+1, df.shape[1]+1)]
    
try:
    icounter # does a exist in the current namespace
except NameError:
    icounter = 0
    
try:
    ptable # does a exist in the current namespace
except NameError:
    ptable = []

try:
    completepvaltable # does a exist in the current namespace
except NameError:
    completepvaltable = []
    
try:
    graphSequence # does a exist in the current namespace
except NameError:
    graphSequence = []    

try:
    gFlag # does a exist in the current namespace
except NameError:
    gFlag = 0   
    
try:
    mleList # does a exist in the current namespace
except NameError:
    mleList = []   
    
try:
    GStatList # does a exist in the current namespace
except NameError:
    GStatList = []   
    
try:
    GStatPValueList # does a exist in the current namespace
except NameError:
    GStatPValueList = []   
    
try:
    rankList # does a exist in the current namespace
except NameError:
    rankList = []   
    
try:
    stopalgo # does a exist in the current namespace
except NameError:
    stopalgo = False   
    
print("gFlag =", gFlag)
e = E[icounter]
# print("e =", e)

# Set G1 to G-e
# For example new G = [[234][124]], set e = [12]

if gFlag == 1:
    X = [lst for lst in G if set(e).issubset(set(lst))]
    G1 = [lst1 for lst1 in G if set(e).issubset(set(lst1)) == False]

#     print("G1 =", G1)

    setB0 = [list(set(lst) - set(e[1:])) for lst in X]
    setB1 = [list(set(lst) - set(e[:-1])) for lst in X]

    G1a = G1.copy()

    if G1a == []:
        for i in range(len(setB0)):
            G1.append(setB0[i])

        for j in range(len(setB1)):
            G1.append(setB1[j])


    for y in setB0:
        if np.sum([set(y).issubset(lst) for lst in G1]) == 0:
            G1.append(y)

    for y in setB1:
        if np.sum([set(y).issubset(lst) for lst in G1]) == 0:
            G1.append(y)
else:
    G1 = G
    
print("G1 =", G1)
print("E =", E)





gFlag = 1
G1 = [[1], [2], [3], [4]]
E = [[3, 4]]


In [358]:
# a2 = np.random.random(10098)
# np.savetxt("foo4.csv", a2, delimiter=",")

In [359]:
# Creating design matrix A and converting contingency table u into vector of counts

df2 = df.copy()
# catvalues = [2,2,2,2]
# df2 = df.copy()
n = df.shape[0]

print("n =", n)

for i in range(len(list(df.columns))):
    dfmin = df[list(df.columns)[i]].min()
    dfmax = df[list(df.columns)[i]].max()
    dfrange = (dfmax - dfmin)/catvalues[i]
#     if catvalues[i] > 2:
#         for j in range(n):
#             jval = df[list(df.columns)[i]][j]
# #             cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if dfmin <= k <= (catvalues[k]-2) else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
#             cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if k <= catvalues[i]-2 else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
#             df2[list(df.columns)[i]][j] = cond.index(True)
            
    for j in df.index.values.tolist():
        jval = df[list(df.columns)[i]][j]
#             cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if dfmin <= k <= (catvalues[k]-2) else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
        cond = [dfmin + k*dfrange <= jval < dfmin + (k+1)*dfrange if k <= catvalues[i]-2 else dfmin + k*dfrange <= jval <= dfmin + (k+1)*dfrange for k in range(catvalues[i])]
        df2[list(df.columns)[i]][j] = cond.index(True)
            
            
#     else:
#         for j in range(size):
#             jval = df[list(df.columns)[i]][j]
#             cond = [dfmin + i*dfrange <= jval < dfmin + (i+1)*dfrange if dfmin <= i <= (catvalues[i]-2) else dfmin + i*dfrange <= jval <= dfmin + (i+1)*dfrange for i in range(catvalues[i])]
#             df2[list(df.columns)[i]][j] = cond.index(True)

# a,b,c = 0,0,1
idx = pd.IndexSlice

def func(lst):
#     dictvalues = {"Pregnancies": 3, "Glucose": 3, "BloodPressure": 3, "SkinThickness": 3, "Insulin": 3, "BMI": 3, "DiabetesPedigreeFunction":3, "Age":3, "Outcome": 2}
#     orderdict = {"Pregnancies": 1, "Glucose": 2, "BloodPressure": 3, "SkinThickness":4, "Insulin":5, "BMI": 6, "DiabetesPedigreeFunction": 7, "Age":  8, "Outcome": 9}
#     order2dict = {1: "Pregnancies", 2: "Glucose", 3: "BloodPressure", 4: "SkinThickness", 5: "Insulin", 6: "BMI", 7: "DiabetesPedigreeFunction", 8: "Age", 9: "Outcome"}

    dictvalues = dict([(df.columns[i], catvalues[i]) for i in range(len(df.columns))])
    orderdict = dict([(df.columns[i], i+1) for i in range(len(df.columns))])
    order2dict = dict([(i+1, df.columns[i]) for i in range(len(df.columns))])
    
    unio = list(set().union(*lst))
    unio2 = sorted(unio)
    mydict = dict(zip(unio2, list(range(len(unio2)))))

    templ = np.zeros(len(unio))
    lst2 = [list(range(1, dictvalues[order2dict[i]]+1)) for i in unio2]
    lst3 = list(itertools.product(*lst2))
    templ = np.zeros(len(lst3))
    emptylist = []

    # Creating the design matrix
    for l in lst:
        for j in list(itertools.product(*[list(range(1, dictvalues[order2dict[i]]+1)) for i in sorted(l)])):
            for k in range(len(lst3)):
                if list(j) == [lst3[k][m] for m in [mydict[n] for n in sorted(l)]]: 
                    templ[k] = 1

            emptylist.append(templ)
            templ = np.zeros(len(lst3))
    
    ltest0 = [i for i in unio2]
    ltest = [df2[j] for j in [order2dict[i] for i in unio2]]
    data_crosstab = pd.crosstab(ltest[:-1], ltest[-1], margins = False, dropna = False) # set margins = True to see margins

    return np.array(emptylist), data_crosstab



n = 5000


In [360]:
matrixA, utemp = func(lst = G1)

print(matrixA)
print(utemp)

# print(matrixA[0,:])
# print(np.sum(matrixA[0,:]))

# print(matrixA.ndim)

matrixA1 = ["{" + str(list(x))[1:-1] + "}" for x in matrixA]
matrixA2 = "{" + str(matrixA1)[1:-1] + "}"
matrixA3 = "{" + str(matrixA)[1:-1] + "}"

tempstr = "matrix {"
for i in range(len(matrixA)-1):
    tempstr = tempstr + "{" + str(list(matrixA[i]))[1:-1] + "}" + ", "
    
tempstr = tempstr + "{" + str(list(matrixA[len(matrixA)-1]))[1:-1] + "}}"

with open("/Users/jc/readmealc", 'w') as f2:
    f2.write(tempstr)
    
f2.close()


[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
  1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.
  0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
  1. 0. 1. 0. 1. 0. 1.

In [361]:
# Calculating rank of design matrix A

# testA = np.array([[1,0,0], [0,1,0], [0,0,1]])
# testA1 = np.array([[1,1,1,0,0,0], [0,0,0,1,1,1], [1,0,0,1,0,0], [0,1,0,0,1,0], [0,0,1,0,0,1]])

rankModel = np.linalg.matrix_rank(matrixA, tol=None, hermitian=False)
# rankModel = np.linalg.matrix_rank(testA1, tol=None, hermitian=False)
print("rankModel =", rankModel)



rankModel = 7


In [362]:
f3 = open("/Users/jc/readmealc", "r")
# print(f3.read())

In [363]:
setup();
restart;
loadPackage "FourTiTwo"
A = value get "/Users/jc/readmealc"
markovBasis = toricMarkov A
print(markovBasis)
"readme5alc" << toString markovBasis << close
copyFile("readme5alc", "/Users/jc/readme5alc")

--initialization text already in file: ~/.profile-Macaulay2
--initialization text already in file: ~/.login-Macaulay2
--initialization text already in file, no changes needed: /Users/jc/.bash_profile
--initialization text already in file, no changes needed: /Users/jc/.zprofile
--initialization text already in file, no changes needed: /Users/jc/.login
--initialization text already in file: ~/.emacs-Macaulay2
--initialization text already in file, no changes needed: /Users/jc/.emacs


| 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  -1 -1 1 |
| 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  -1 -1 1  0  0 |
| 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  -1 0  0  -1 1 |
| 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  -1 -1 1  0  0  0  0 |
| 0 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 

In [364]:
pythonf = open("/Users/jc/readme5alc", "r")
x = pythonf.read()

print(G1)

# print(markovBasis)
# restart;
# print(A)
# // "readme5" << toString markovBasis << close

[[1], [2], [3], [4]]


In [365]:
if G1 != [[i+1 for i in range(df.shape[1])]]:
    new_string = x[7:].replace("{", "[" )
    new_string2 = new_string.replace("}", "]" )

    y = json.loads(new_string2)

    print(y)
    # print(type(y))

    B = np.array(y)
    print("B =", B)
    # print(B.shape)
else:
    B = []
    
# print(B)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, -1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, -1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, -1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, -1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1, 0, 0, 0, -1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, -1, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, -1, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0, 0, -1, 1


B = [[ 0  0  0 ... -1 -1  1]
 [ 0  0  0 ...  1  0  0]
 [ 0  0  0 ...  0 -1  1]
 ...
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  1  0]]


In [366]:
A,u = func(lst = G1)
u = u.to_numpy()
u = u.tolist()
u = np.array([item for items in u for item in items])

# u = u*20 + 1

A1 = A.flatten()

print("A =", A)
print("u =", u)
# print("u1 =", u1)

print(np.sum(u))

A = [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
  1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
  1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0.
  0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.
  0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.]
 [1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.
  1. 0. 1. 0. 1. 0

In [367]:
# BMI vs diabetes

import random
import math
import pandas as pd
import numpy as np
from numba import jit, prange
from scipy.special import factorial

A,u = func(lst = G1)
u = u.to_numpy()
u = u.tolist()
u = np.array([item for items in u for item in items])

if G1 != [[i+1 for i in range(df.shape[1])]]:
    Btemp = B
    B = np.concatenate((Btemp,-Btemp))

# @jit(nopython=True, parallel=True, fastmath=True)
def calcs(numsamples, A, u, B):
    avert = np.array([A[:,i] for i in range(A.shape[1])])
    a = np.zeros(numsamples)
    a3 = np.zeros(numsamples)
    for i7 in prange(numsamples):
        a[i7] = np.random.rand()
        a3[i7] = np.random.rand()

    randlist1 = a.copy()
    randlist2 = a3.copy()
    chilist = np.zeros(numsamples)
    
    p1 = np.sum(u)*np.ones(u.shape[0])/u.shape[0]  
    
    tol1 = 0.001
    adot = np.zeros(A.shape[0])
    pcalc = np.zeros(A.shape[0])
    p1tempcalc = np.zeros(A.shape[0])
    ptempcalc = np.zeros(A.shape[0])

    
    adot1 = np.dot(A,u)
    
    while np.sum(np.absolute(np.dot(A,p1) - adot1)) > tol1:
        p1temp = p1.copy()
        p1 = np.array([p1temp[j] * (np.prod(np.power(adot1, avert[j])) / np.prod(np.power(np.dot(A,p1temp), avert[j])))**(1/np.sum(A[0,:])) for j in range(u.shape[0])])      
    
    chip = np.sum(np.array([((u[a1] - p1[a1])**2)/p1[a1] for a1 in range(u.shape[0])]))
    
    if G1 != [[i+1 for i in range(df.shape[1])]]:
        v = u
        for m in range(numsamples):
            w = B[int(len(B) * randlist1[m])]
            w1 = w.copy()
            
            if np.min(v+w1) < 0:
                v = v
            else:       
                qprod = np.array([v[i] / (v[i] + w1[i]) if v[i] != v[i] + w1[i] and v[i] != 0 and (v[i] + w1[i]) != 0 else 1 for i in range(len(v))])
                q = np.prod(qprod)

                if q >= 1:
                    v = v + w1
                elif randlist2[m] <= q:
                    v = v + w1

            p = np.sum(u)*np.ones(u.shape[0])/u.shape[0]
            adot = np.dot(A,v)
            
            while np.sum(np.absolute(np.dot(A,p) - adot)) > tol1:
                ptemp = p.copy()
                p = np.array([ptemp[j9] * (np.prod(np.power(adot, avert[j9])) / np.prod(np.power(np.dot(A,ptemp), avert[j9])))**(1/np.sum(A[0,:])) for j9 in range(u.shape[0])])

            chitemp = np.array([((v[b1] - p[b1])**2)/p[b1] for b1 in range(u.shape[0])])
            if np.sum(chitemp) >= chip:
                chilist[m] = 1
    
        return np.sum(chilist)/numsamples, p1
    
    else:
        return 1, p1

# if G1 != [[1,2,3,4]]:
#     pval = calcs(100, A, np.array(u), B)[0]
# else:
#     pva1l = 1
    
pval, mleG = calcs(1000, A, np.array(u), B)

print("mleG =", mleG)



mleG = [140.86656926 133.94142388 152.80143223 145.28955671 141.91195881
 134.935421   136.846605   130.1190852  148.44087813 141.14337193
 137.86216186 131.08471624 141.03406777 134.10068798 152.98312198
 145.4623144  142.08070034 135.09586703 137.96670988 131.18412456
 149.65588343 142.29864633 138.99057918 132.1576594  134.02950004
 127.44047201 145.38509509 138.23781433 135.02415079 128.3862247
 138.13076028 131.34011008 149.83383294 142.46784766 139.15584703
 132.31480251]


In [368]:
# Now calculate the G^2 statistic
from scipy.stats.distributions import chi2

if G1 != [[i+1 for i in range(df.shape[1])]]:
#     print("mleG =", mleG/np.sum(mleG))
#     print("mleList[-1] =", mleList[-1]/np.sum(mleList[-1]))
    
    mleG1 = mleG
#     mleG1 = mleG/np.sum(mleG)
    mleList1 = mleList[-1]
#     mleList1 = mleList[-1]/np.sum(mleList[-1])
    
    print(mleList1)
    print(mleG1)

    
#     mleG1[i]
#     mleList1[i]
    
    
#     GSquared = np.sum([mleG[i] * np.log(mleG[i] / mleList[-1][i]) for i in range(len(mleG))])
    GSquared = 2*np.sum([mleList1[i] * np.log(mleList1[i] / mleG1[i]) for i in range(len(mleG1))])
    print("G squared =", GSquared)
    GStatList.append(GSquared)
    rankDiff = rankList[-1] - rankModel
    
    print("rankDiff =", rankDiff)
    
    pvalG = chi2.sf(GSquared, rankDiff)
    
    if pvalG * len(E) > 1:
        pvalGBonf = 1
    else:
        pvalGBonf = pvalG * len(E)
    
#     pvalGBonf = pvalG * len(E)
    
    
    
    print("pvalG =", pvalG)
else:
    mleList.append(mleG)
    rankList.append(rankModel)



[142.75738303 132.05061292 154.99368852 143.09728045 137.82887014
 139.01851124 138.68346369 128.28223657 150.57057728 139.01366133
 133.89559757 135.05128944 142.92712966 132.20762859 155.17798481
 143.26743124 137.99275649 139.18381214 139.81860238 129.33224013
 151.80301324 140.15150272 134.99154707 136.1566984  135.82854809
 125.64143897 147.47095546 136.15194834 131.13924421 132.2711452
 139.98485463 129.4860235  151.98351564 140.31815081 135.15205968
 136.31859644]
[140.86656926 133.94142388 152.80143223 145.28955671 141.91195881
 134.935421   136.846605   130.1190852  148.44087813 141.14337193
 137.86216186 131.08471624 141.03406777 134.10068798 152.98312198
 145.4623144  142.08070034 135.09586703 137.96670988 131.18412456
 149.65588343 142.29864633 138.99057918 132.1576594  134.02950004
 127.44047201 145.38509509 138.23781433 135.02415079 128.3862247
 138.13076028 131.34011008 149.83383294 142.46784766 139.15584703
 132.31480251]
G squared = 2.1040535447568764
rankDiff = 2
pval

In [369]:
print("G1 =", G1)

if G1 != [[i+1 for i in range(df.shape[1])]]: 
    ptable.append([G1, e, pval, pvalG, rankModel, mleG, pvalGBonf])
    completepvaltable.append([G1, e, pval, pvalG, '', '', pvalGBonf])

#     print(ptable)

    if icounter == (len(E)-1):
        # From the p-value table, select the edge with the highest p-value above alpha = 0.05
        ptable1 = [lst[6] for lst in ptable]
#         ptable1 = [lst[6] for lst in ptable]

#         if ptable[ptable1.index(max(ptable1))][2] > 0.05:
        if ptable[ptable1.index(max(ptable1))][6] > 0.05:    
            print("edge removed is", ptable[ptable1.index(max(ptable1))][1])

#             graphSequence.append([ptable[ptable1.index(max(ptable1))][1], ptable[ptable1.index(max(ptable1))][0], ptable[ptable1.index(max(ptable1))][2]])
            graphSequence.append([ptable[ptable1.index(max(ptable1))][1], ptable[ptable1.index(max(ptable1))][0], ptable[ptable1.index(max(ptable1))][2], ptable[ptable1.index(max(ptable1))][3], ptable[ptable1.index(max(ptable1))][6]])
            completepvaltable.append(['', '', '', '', ptable[ptable1.index(max(ptable1))][1], ptable[ptable1.index(max(ptable1))][0], ''])

            E.remove(ptable[ptable1.index(max(ptable1))][1])
            print("new E =", E)
            icounter = 0
            print("G =", ptable[ptable1.index(max(ptable1))][0])
            G = ptable[ptable1.index(max(ptable1))][0]
            rankList.append(ptable[ptable1.index(max(ptable1))][4])
            mleList.append(ptable[ptable1.index(max(ptable1))][5])
            ptable = []

            if E == []:
                print("Stop algorithm")
                print("Resulting graph is", G)
                print("Complete p-value table =", completepvaltable)
                stopalgo = True

        else:
            print("Complete p-value table =", completepvaltable)
            print("Stop algorithm")
            print("Resulting graph is", G)
            stopalgo = True
            # print(ptable)

    else:
        icounter += 1
    #     %run sbf.ipynb

    # print(ptable)


G1 = [[1], [2], [3], [4]]
edge removed is [3, 4]
new E = []
G = [[1], [2], [3], [4]]
Stop algorithm
Resulting graph is [[1], [2], [3], [4]]
Complete p-value table = [[[[1, 3, 4], [2, 3, 4]], [1, 2], 0.892, 0.7335598170606015, '', '', 1], [[[1, 2, 4], [2, 3, 4]], [1, 3], 0.958, 0.9920339864056613, '', '', 1], [[[1, 2, 3], [2, 3, 4]], [1, 4], 0.977, 0.9893616036562292, '', '', 1], [[[1, 2, 4], [1, 3, 4]], [2, 3], 0.854, 0.95431298581412, '', '', 1], [[[1, 2, 3], [1, 3, 4]], [2, 4], 0.992, 0.9965682319362347, '', '', 1], [[[1, 2, 3], [1, 2, 4]], [3, 4], 0.999, 0.9206554174220286, '', '', 1], ['', '', '', '', [1, 2], [[1, 3, 4], [2, 3, 4]], ''], [[[2, 3, 4], [1, 4]], [1, 3], 0.551, 0.8854142400728403, '', '', 1], [[[2, 3, 4], [1, 3]], [1, 4], 0.989, 0.6087963801279486, '', '', 1], [[[1, 3, 4], [2, 4]], [2, 3], 0.955, 0.6954252512903116, '', '', 1], [[[1, 3, 4], [2, 3]], [2, 4], 0.968, 0.8633399926764409, '', '', 1], [[[1, 3], [2, 3], [1, 4], [2, 4]], [3, 4], 0.842, 0.6791334606222124, '', 

In [370]:
if G1 != [[i+1 for i in range(df.shape[1])]]:
#     dfGsequence = pd.DataFrame({'Edge deleted': [i[0] for i in graphSequence], 'Resulting graph': [i[1] for i in graphSequence], 'p-value': [i[2] for i in graphSequence]})
    dfGsequence = pd.DataFrame({'Edge deleted': [i[0] for i in graphSequence], 'Resulting graph': [i[1] for i in graphSequence], 'p-value G^2': [i[3] for i in graphSequence], 'p-value G^2 Bonf': [i[4] for i in graphSequence]})



    dfpvalue = pd.DataFrame({'G-e': [i[0] for i in completepvaltable], 'e': [i[1] for i in completepvaltable], 'p-value G^2': [i[3] for i in completepvaltable], 'Bonf p-value G^2': [i[6] for i in completepvaltable], 'Delete edge': [i[4] for i in completepvaltable], 'Resulting graph': [i[5] for i in completepvaltable]})

    print(dfpvalue.to_string())
    print(dfGsequence.to_string())
else:
    gFlag = 1
    
print(gFlag)

                                 G-e       e p-value G^2 Bonf p-value G^2 Delete edge         Resulting graph
0             [[1, 3, 4], [2, 3, 4]]  [1, 2]     0.73356                1                                    
1             [[1, 2, 4], [2, 3, 4]]  [1, 3]    0.992034                1                                    
2             [[1, 2, 3], [2, 3, 4]]  [1, 4]    0.989362                1                                    
3             [[1, 2, 4], [1, 3, 4]]  [2, 3]    0.954313                1                                    
4             [[1, 2, 3], [1, 3, 4]]  [2, 4]    0.996568                1                                    
5             [[1, 2, 3], [1, 2, 4]]  [3, 4]    0.920655                1                                    
6                                                                              [1, 2]  [[1, 3, 4], [2, 3, 4]]
7                [[2, 3, 4], [1, 4]]  [1, 3]    0.885414                1                                    
8         

In [371]:
if stopalgo == True:
    dfpvalue.to_csv('fresults.csv', index=False)
    xd = pd.read_csv('fresults.csv', sep=',')
    print(xd.to_string())

                                 G-e       e  p-value G^2  Bonf p-value G^2 Delete edge         Resulting graph
0             [[1, 3, 4], [2, 3, 4]]  [1, 2]     0.733560          1.000000         NaN                     NaN
1             [[1, 2, 4], [2, 3, 4]]  [1, 3]     0.992034          1.000000         NaN                     NaN
2             [[1, 2, 3], [2, 3, 4]]  [1, 4]     0.989362          1.000000         NaN                     NaN
3             [[1, 2, 4], [1, 3, 4]]  [2, 3]     0.954313          1.000000         NaN                     NaN
4             [[1, 2, 3], [1, 3, 4]]  [2, 4]     0.996568          1.000000         NaN                     NaN
5             [[1, 2, 3], [1, 2, 4]]  [3, 4]     0.920655          1.000000         NaN                     NaN
6                                NaN     NaN          NaN               NaN      [1, 2]  [[1, 3, 4], [2, 3, 4]]
7                [[2, 3, 4], [1, 4]]  [1, 3]     0.885414          1.000000         NaN                 

In [372]:
# [[i+1 for i in range(df.shape[1])]]